In [1]:
using CSV, DataFrames, NamedArrays, Combinatorics

In [12]:
import Pkg; Pkg.add("Combinatorics")
using Combinatorics

    Updating registry at `C:\Users\Valeria\.julia\registries\General`
   Resolving package versions...
    Updating `C:\Users\Valeria\.julia\environments\v1.6\Project.toml`
  [861a8166] + Combinatorics v1.0.2
  No Changes to `C:\Users\Valeria\.julia\environments\v1.6\Manifest.toml`


In [2]:
data = CSV.read("BM93e3.csv", DataFrame)

,blk,A,B,C,D,E,F,G,H,y
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,-1,-1,-1,-1,1,1,1,-1,1,14.0
2,-1,1,-1,-1,-1,-1,1,1,1,16.8
3,-1,-1,1,-1,-1,1,-1,1,1,15.0
4,-1,1,1,-1,1,-1,-1,-1,1,15.4
5,-1,-1,-1,1,1,-1,-1,1,1,27.6
6,-1,1,-1,1,-1,1,-1,-1,1,24.0
7,-1,-1,1,1,-1,-1,1,-1,1,27.4
8,-1,1,1,1,1,1,1,1,1,22.6
9,-1,1,1,1,-1,-1,-1,1,-1,22.3


In [3]:
X = data[1:16, [1, 2, 4, 6, 9]]
y = data[1:16, 10]

16-element Vector{Float64}:
 14.0
 16.8
 15.0
 15.4
 27.6
 24.0
 27.4
 22.6
 22.3
 17.1
 21.5
 17.5
 15.9
 21.9
 16.7
 20.3

In [4]:
p_mod = [0.2356,0.2356,0.2356,0.2356,0.0566]

5-element Vector{Float64}:
 0.2356
 0.2356
 0.2356
 0.2356
 0.0566

In [5]:
fac_mod = NamedArray([2 3 4 0; 1 3 4 0; 1 2 3 0; 1 2 4 0; 1 2 3 4])
setnames!(fac_mod, ["f1", "f2", "f3", "f4"], 2)  
fac_mod

5×4 Named Matrix{Int64}
A ╲ B │ f1  f2  f3  f4
──────┼───────────────
1     │  2   3   4   0
2     │  1   3   4   0
3     │  1   2   3   0
4     │  1   2   4   0
5     │  1   2   3   4

In [6]:
# Quiero arreglar esta construcción, se ve horrible
Xcand = NamedArray([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1] [-1,-1,-1,-1,1,1,1,1,-1,-1,-1,-1,1,1,1,1] [-1,-1,1,1,-1,-1,1,1,-1,-1,1,1,-1,-1,1,1] [-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1] [-1,1,1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,1,-1]])
setnames!(Xcand, ["blk", "f1", "f2", "f3", "f4"], 2)
Xcand

16×5 Named Matrix{Int64}
A ╲ B │ blk   f1   f2   f3   f4
──────┼────────────────────────
1     │   1   -1   -1   -1   -1
2     │   1   -1   -1    1    1
3     │   1   -1    1   -1    1
4     │   1   -1    1    1   -1
5     │   1    1   -1   -1    1
6     │   1    1   -1    1   -1
7     │   1    1    1   -1   -1
8     │   1    1    1    1    1
9     │   1   -1   -1   -1    1
10    │   1   -1   -1    1   -1
11    │   1   -1    1   -1   -1
12    │   1   -1    1    1    1
13    │   1    1   -1   -1   -1
14    │   1    1   -1    1    1
15    │   1    1    1   -1    1
16    │   1    1    1    1   -1

In [16]:
# To drop columns
dropcol(M::NamedArray, j) = M[:, deleteat!(collect(axes(M, 2)), j)]
dropcol(M::DataFrame, j) = M[:, deleteat!(collect(axes(M, 2)), j)]

dropcol (generic function with 2 methods)

In [116]:
nMod = 5
max_int = 2
g = 2

2

In [18]:
# 1
n = size(y, 1)
fac = size(X, 2) - 1
k = size(fac_mod, 2)

# Todavia no estoy segura si los dejare como listas o mejor como vectores
Si = []
Xi = Array{Array{Int64}}[]
betai = []
gammai = Array{Array{Int64}}[]
efectos = Vector{Vector{Int8}}(undef,nMod)

models = zeros(nMod, fac)
for i = 1:nMod
    aux = fac_mod[i, :]
    filter!(e -> e != 0, aux)
    models[i, aux] .= 1
end

Xfac = dropcol(X, 1)
Xc = dropcol(Xcand, 1)

16×4 Named Matrix{Int64}
A ╲ B │ f1  f2  f3  f4
──────┼───────────────
1     │ -1  -1  -1  -1
2     │ -1  -1   1   1
3     │ -1   1  -1   1
4     │ -1   1   1  -1
5     │  1  -1  -1   1
6     │  1  -1   1  -1
7     │  1   1  -1  -1
8     │  1   1   1   1
9     │ -1  -1  -1   1
10    │ -1  -1   1  -1
11    │ -1   1  -1  -1
12    │ -1   1   1   1
13    │  1  -1  -1  -1
14    │  1  -1   1   1
15    │  1   1  -1   1
16    │  1   1   1  -1

In [19]:
# 2
models = NamedArray(models)
if max_int > 1
    comb = hcat(collect(combinations(1:fac,2))...)
    mat = zeros(Int8, size(models, 1), size(comb, 2))
    for j = 1:size(comb, 2)
        fac1 = comb[1, j]
        fac2 = comb[2, j]
        aux = models[:, fac1] + models[:, fac2] .== 2
        filter!(e -> e != 0, aux)
        cols = names(aux, 1)
        cols = parse.(Int8, cols)
        mat[cols, j] .= 1
        Xfac = hcat(Xfac, Xfac[:, fac1].*Xfac[:, fac2], makeunique = true)    
        Xc = hcat(Xc, Xc[:, fac1].*Xc[:, fac2])
    end
    mat = NamedArray(mat)
    models = hcat(models, mat)
end


5×10 Named Matrix{Float64}
A ╲ hcat │   1    2    3    4    5    6    7    8    9   10
─────────┼─────────────────────────────────────────────────
1        │ 0.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0
2        │ 1.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0
3        │ 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
4        │ 1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  0.0
5        │ 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [20]:
# 3
if max_int > 2
    comb = hcat(collect(combinations(1:fac,3))...)
    mat = zeros(Int8, size(models, 1), size(comb, 2))
    for j = 1:size(comb, 2)
        fac1 = comb[1, j]
        fac2 = comb[2, j]
        fac3 = comb[3, j]
        aux = models[:, fac1] + models[:, fac2] + models[:, fac3] .== 3
        filter!(e -> e != 0, aux)
        cols = names(aux, 1)
        cols = parse.(Int8, cols)
        mat[cols, j] .= 1
        Xfac = hcat(Xfac, Xfac[:, fac1].*Xfac[:, fac2].*Xfac[:, fac3], makeunique = true)    
        Xc = hcat(Xc, Xc[:, fac1].*Xc[:, fac2].*Xc[:, fac3])
    end
    mat = NamedArray(mat)
    models = hcat(models, mat)
end

Xfac = hcat(X[:, 1], Xfac, makeunique = true)
Xc = hcat(X[:, 1], Xc)
models = hcat(ones(nMod), models)

5×11 Matrix{Float64}:
 1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0
 1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [ ]:
# 4
models = NamedArray(models)
for i=1:nMod
    aux = models[1, :] .== 1
    filter!(e -> e != 0, aux)
    cols = names(aux, 1)
    cols = parse.(Int8, cols)
    efectos[i] = cols
    tam = length(efectos[i])
    push!(Xi, insertcols!(Xfac[:, efectos[i]], size(Xfac[:, efectos[i]], 2) + 1,  :efectos => ones(n) , makeunique=true))
    
    mat = zeros(Int8, tam + 1, tam + 1)
    if size(mat, 1) > 1
    # Tengo que hacer este for mas eficiente
    coord = hcat(2:size(mat, 1), 2:size(mat, 1))
    for k=1:(size(mat, 1) - 1)
        mat[coord[k, 1], coord[k, 2]] = 1
    end
        #esto no funcionaaa
    push!(gammai, (1 / g^2)*mat)

In [120]:
gammai = Array{Array{Int64}}[]

Array{Array{Int64, N} where N, N} where N[]

In [122]:
(1 / g^2)*mat

8×8 Matrix{Float64}:
 0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0
 0.0  0.25  0.0   0.0   0.0   0.0   0.0   0.0
 0.0  0.0   0.25  0.0   0.0   0.0   0.0   0.0
 0.0  0.0   0.0   0.25  0.0   0.0   0.0   0.0
 0.0  0.0   0.0   0.0   0.25  0.0   0.0   0.0
 0.0  0.0   0.0   0.0   0.0   0.25  0.0   0.0
 0.0  0.0   0.0   0.0   0.0   0.0   0.25  0.0
 0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.25

In [121]:
push!(gammai, (1 / g^2)*mat)

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mFloat64[39m[0m to an object of type 
[0m  [91mArray{Int64, N} where N[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::AbstractArray[39m) where T<:Array at array.jl:532
[0m  convert(::Type{T}, [91m::LinearAlgebra.Factorization[39m) where T<:AbstractArray at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\factorization.jl:58
[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractArray at abstractarray.jl:14
[0m  ...

In [115]:
Xi[i]

,x1,C,E,H,x1_3,x1_4,x1_5,efectos
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,-1,-1,1,1,-1,-1,1,1.0
2,-1,-1,-1,1,1,-1,-1,1.0
3,-1,-1,1,1,-1,-1,1,1.0
4,-1,-1,-1,1,1,-1,-1,1.0
5,-1,1,-1,1,-1,1,-1,1.0
6,-1,1,1,1,1,1,1,1.0
7,-1,1,-1,1,-1,1,-1,1.0
8,-1,1,1,1,1,1,1,1.0
9,-1,1,-1,-1,-1,-1,1,1.0
